# Лабораторная работа №3
## Тема: Метод проекции градиента

In [1]:
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [2]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)

#### Целевая функция
$3x+y+2z \rightarrow min$ 

In [3]:
def f(x):
    y = 3*x[0] + x[1] + 2*x[2]
    return y

#### Условия
$2x^2+y^2+3z^2\le1$

In [4]:
ball_coefs = np.array([2, 1, 3])
ball_center = np.array([0, 0, 0])
ball_radius = 1

#### Производные функции

In [5]:
def df(x, h_step=0.01):
    h_matr = h_step * np.eye(x.shape[0])
    y = np.array([(f(x+h) - f(x-h))/(2*h_step) for h in h_matr])
    return y

In [6]:
def d2f(x, h_step=0.01):
    h_matr = h_step * np.eye(x.shape[0])
    y = np.array([(df(x+h) - df(x-h))/(2*h_step) for h in h_matr])
    return y

#### Метод проекции

In [7]:
def ball_proj(x, c, r):
    if np.linalg.norm(x - c) > r:
        proj_x = c + (x - c)/np.linalg.norm(x - c)*r
    else:
        proj_x = x
    return proj_x

#### Градиентный спуск с постоянным шагом

In [8]:
def run_const(start_data, error=1e-6):
    x_arr = copy(start_data)
    alpha = 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [9]:
size = 10
start = [2*size*(np.random.random(3) - 0.5)]
e = 1e-6

In [10]:
first = run_const(start, e)

In [23]:
first

[array([3.7565602 , 8.8047653 , 7.47499854]),
 array([0.08678742, 0.63307931, 0.76920643]),
 array([-0.8853808 , -0.07885244, -0.45813004]),
 array([-0.78112112, -0.15336697, -0.60525066]),
 array([-0.75407064, -0.16264621, -0.63633614]),
 array([-0.74791304, -0.16378769, -0.64327263]),
 array([-0.74653715, -0.16391575, -0.64483635]),
 array([-0.74622991, -0.1639265 , -0.64518913]),
 array([-0.74616117, -0.16392635, -0.64526867]),
 array([-0.74614576, -0.16392596, -0.64528659]),
 array([-0.74614231, -0.16392581, -0.64529062]),
 array([-0.74614153, -0.16392577, -0.64529153]),
 array([-0.74614136, -0.16392576, -0.64529173])]

#### Быстрейший спуск

In [12]:
def run_fastest(start_data, error=1e-6):
    x_arr = copy(start_data)
    alpha = 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        A = d2f(x)
        alpha = (h @ h)/(A @ h @ h)
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [13]:
# second = run_fastest(start, e)

In [24]:
# second

#### Градиентный спуск с дроблением шага

In [19]:
def run_shredded(start_data, error=1e-6):
    x_arr = copy(start_data)
    coef = 0.5
    alpha= 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        while f(x - alpha*h) >= f(x):
            alpha = coef * alpha
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [20]:
third = run_shredded(start, e)

In [22]:
third

[array([3.7565602 , 8.8047653 , 7.47499854]),
 array([0.08678742, 0.63307931, 0.76920643]),
 array([-0.8853808 , -0.07885244, -0.45813004]),
 array([-0.78112112, -0.15336697, -0.60525066]),
 array([-0.75407064, -0.16264621, -0.63633614]),
 array([-0.74791304, -0.16378769, -0.64327263]),
 array([-0.74653715, -0.16391575, -0.64483635]),
 array([-0.74622991, -0.1639265 , -0.64518913]),
 array([-0.74616117, -0.16392635, -0.64526867]),
 array([-0.74614576, -0.16392596, -0.64528659]),
 array([-0.74614231, -0.16392581, -0.64529062]),
 array([-0.74614153, -0.16392577, -0.64529153]),
 array([-0.74614136, -0.16392576, -0.64529173])]